In [1]:
import pandas as pd
import os
#from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit

In [21]:
def merge_chunks(right_on_list, input_file, output_file, rename_columns={}, drop_columns=[], chunksize=10**7, how='left', index=False, usecols=None):
    first_time = True
    size = 0
    i = 0
    
    f = open(output_file, 'w')

    for left in pd.read_csv(input_file, usecols=usecols, chunksize=chunksize):
        for right, on in right_on_list:
            left = left.merge(right, how=how, on=on)
            
        if len(rename_columns)>0:
            left.rename(columns=rename_columns, inplace=True)
            
        if len(drop_columns)>0:
            left.drop(drop_columns, axis=1, inplace = True)
    
        if first_time:
            left.to_csv(f, index = index, header=True)
            f.close()
            f = open(output_file, 'a')
            first_time = False
        else:
            left.to_csv(f, index = index, header=False)
    
        size+=chunksize
        i+=1
        print i, size
            
    print 'Done'
    
    f.close()

## Load / preview common tables to be joined

In [3]:
events_filename = "../generated/final/events.csv"

In [4]:
events = pd.read_csv(events_filename, nrows=10)
events.head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1        2657      1271490     185255             1.0       2.0    1     7   
2        6344       461545     434576             1.0       2.0    1     7   
3        7039      1793073     482930             1.0       1.0    1     7   
4       11527      1518592     800258             3.0       2.0    1     7   

    geo  
0  2765  
1  2765  
2  2765  
3  2765  
4  2765

In [5]:
adsPerDisplay = pd.read_csv("../generated/adsPerDisplay.csv")
adsPerDisplay.count()

display_id       23120126
adsPerDisplay    23120126
dtype: int64

## Join training

### Step 1

In [6]:
clicks_train = pd.read_csv("../download/clicks_train.csv")
clicks_train.count()

display_id    87141731
ad_id         87141731
clicked       87141731
dtype: int64

In [7]:
right_on_list = [(clicks_train, 'display_id'), (adsPerDisplay, 'display_id')] 
input_file = events_filename
output_file = "../generated/final/events_clicks_train.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='inner'

In [8]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
Done


In [9]:
pd.read_csv(output_file, nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1           8      1330329        638             1.0       2.0    1     7   
2           8      1330329        638             1.0       2.0    1     7   
3           8      1330329        638             1.0       2.0    1     7   
4        2657      1271490     185255             1.0       2.0    1     7   

    geo   ad_id  clicked  adsPerDisplay  
0  2765   95724        0              4  
1  2765  175694        0              4  
2  2765  280430        1              4  
3  2765  329774        0              4  
4  2765   70081        0              4

### Step 2

Can be run after feature_base_3

In [5]:
events_CTR_train = pd.read_csv("../generated/final/events_CTR_train.csv")
events_CTR_train.count()

display_id        87141731
ad_id             87141731
clicksPerShows    87141731
dtype: int64

In [ ]:
right_on_list = [(events_CTR_train, ['display_id', 'ad_id'])] 
input_file = "../generated/final/events_clicks_train.csv"
output_file = "../generated/final/events_clicks_CTR_train.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='inner'

In [7]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
6 30000000
7 35000000
8 40000000
9 45000000
10 50000000
11 55000000
12 60000000
13 65000000
14 70000000
15 75000000
16 80000000
17 85000000
18 90000000
Done


In [4]:
pd.read_csv(output_file, nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1           8      1330329        638             1.0       2.0    1     7   
2           8      1330329        638             1.0       2.0    1     7   
3           8      1330329        638             1.0       2.0    1     7   
4        2657      1271490     185255             1.0       2.0    1     7   

    geo   ad_id  clicked  adsPerDisplay  clicksPerShows  
0  2765   95724        0              4        0.034785  
1  2765  175694        0              4        0.202327  
2  2765  280430        1              4        0.256853  
3  2765  329774        0              4        0.052426  
4  2765   70081        0              4        0.150145

### Step 3

In [12]:
ads = pd.read_csv("../download/promoted_content.csv", names=['ad_id', 'addoc_id', 'c', 'a'], header=0, usecols=['ad_id', 'addoc_id'])
ads.head()

ad_id  addoc_id
0      1      6614
1      2    471467
2      3      7692
3      4    471471
4      5    471472

In [25]:
right_on_list = [(ads, ['ad_id'])] 
input_file = "../generated/final/events_clicks_CTR_train.csv"
output_file = "../generated/final/events_clicks_CTR_addocs_train.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='left'

In [26]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
6 30000000
7 35000000
8 40000000
9 45000000
10 50000000
11 55000000
12 60000000
13 65000000
14 70000000
15 75000000
16 80000000
17 85000000
18 90000000
Done


In [27]:
pd.read_csv(output_file, nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0           8      1330329        638             1.0       2.0    1     7   
1           8      1330329        638             1.0       2.0    1     7   
2           8      1330329        638             1.0       2.0    1     7   
3           8      1330329        638             1.0       2.0    1     7   
4        2657      1271490     185255             1.0       2.0    1     7   

    geo   ad_id  clicked  adsPerDisplay  clicksPerShows  addoc_id  
0  2765   95724        0              4        0.034785   1136820  
1  2765  175694        0              4        0.202327   1392479  
2  2765  280430        1              4        0.256853   1601512  
3  2765  329774        0              4        0.052426   1652347  
4  2765   70081        0              4        0.150145    933594

## Join testing

### Step 1

In [ ]:
clicks_test = pd.read_csv("../download/clicks_test.csv")
clicks_test.count()

In [ ]:
clicks_test.head()

In [ ]:
ids = clicks_test.display_id

In [ ]:
prev = 0
for nxt in ids:
    if nxt<prev:
        print nxt
        break
    prev = nxt    

In [ ]:
right_on_list = [(clicks_test, 'display_id'), (adsPerDisplay, 'display_id')] 
input_file = events_filename
output_file = "../generated/final/events_clicks_test.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='inner'

In [ ]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

In [ ]:
pd.read_csv(output_file, nrows=5).head()

### Step 2

In [9]:
events_CTR_test = pd.read_csv("../generated/final/events_CTR_test.csv")
events_CTR_test.count()

display_id        32225162
ad_id             32225162
clicksPerShows    32225162
dtype: int64

In [18]:
right_on_list = [(events_CTR_test, ['display_id', 'ad_id'])] 
input_file = "../generated/final/events_clicks_test.csv"
output_file = "../generated/final/events_clicks_CTR_test.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='inner'

In [19]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
6 30000000
7 35000000
Done


In [20]:
pd.read_csv(output_file, nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0    16874904      1669669     120418             1.0       2.0    1     7   
1    16874904      1669669     120418             1.0       2.0    1     7   
2    16874904      1669669     120418             1.0       2.0    1     7   
3    16874904      1669669     120418             1.0       2.0    1     7   
4    16874904      1669669     120418             1.0       2.0    1     7   

    geo   ad_id  adsPerDisplay  clicksPerShows  
0  2765   51910              8        0.095014  
1  2765  151689              8        0.170815  
2  2765  156459              8        0.015815  
3  2765  165380              8        0.409523  
4  2765  256392              8        0.078487

### Step 3

In [12]:
ads = pd.read_csv("../download/promoted_content.csv", names=['ad_id', 'addoc_id', 'c', 'a'], header=0, usecols=['ad_id', 'addoc_id'])
ads.head()

ad_id  addoc_id
0      1      6614
1      2    471467
2      3      7692
3      4    471471
4      5    471472

In [22]:
right_on_list = [(ads, ['ad_id'])] 
input_file = "../generated/final/events_clicks_CTR_test.csv"
output_file = "../generated/final/events_clicks_CTR_addocs_test.csv"   
rename_columns={}
drop_columns=[]
chunksize=5 * 10**6
how='left'

In [23]:
merge_chunks(right_on_list, input_file, output_file, rename_columns, drop_columns, chunksize, how)

1 5000000
2 10000000
3 15000000
4 20000000
5 25000000
6 30000000
7 35000000
Done


In [24]:
pd.read_csv(output_file, nrows=5).head()

display_id  document_id  timestamp  traffic_source  platform  day  hour  \
0    16874904      1669669     120418             1.0       2.0    1     7   
1    16874904      1669669     120418             1.0       2.0    1     7   
2    16874904      1669669     120418             1.0       2.0    1     7   
3    16874904      1669669     120418             1.0       2.0    1     7   
4    16874904      1669669     120418             1.0       2.0    1     7   

    geo   ad_id  adsPerDisplay  clicksPerShows  addoc_id  
0  2765   51910              8        0.095014    969482  
1  2765  151689              8        0.170815   1262645  
2  2765  156459              8        0.015815    688201  
3  2765  165380              8        0.409523   1378725  
4  2765  256392              8        0.078487   1626651

## Validation splits

In [3]:
train_filename = "../generated/final/events_clicks_CTR_train.csv"

In [26]:
displays = pd.read_csv("../download/events.csv", usecols=['display_id'])
displays.count()

display_id    23120126
dtype: int64

In [5]:
#kf = KFold(n_splits=5)
#train, test = kf.split(displays).next()
ss = ShuffleSplit(n_splits=1, test_size=0.25, random_state=0)
train_displays, test_displays = ss.split(displays).next()

print len(train_displays), len(test_displays)  #17340094 5780032

17340094 5780032


In [27]:
displays = displays.iloc[test_displays].copy()
displays['mark'] = [1]*len(displays)
displays.count()

display_id    5780032
dtype: int64

In [30]:
train = pd.read_csv(train_filename, usecols=['display_id'])
train.count()

display_id    87141731
dtype: int64

In [31]:
train_split = pd.merge(train, displays, on='display_id', how='left')
train_split.count()

display_id    87141731
mark          21786490
dtype: int64

In [32]:
train_index = train_split[train_split['mark'].isnull()].index
test_index = train_split[train_split['mark'].notnull()].index
len(train_index), len(test_index)

In [41]:
pd.DataFrame({'index':train_index}).to_csv("../generated/final/train_index.csv", index=False)
pd.DataFrame({'index':test_index}).to_csv("../generated/final/test_index.csv", index=False)

In [ ]:
pd.read_csv("../generated/final/train_index.csv", nrows=10).head(10)